In [2]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import sqlite3

conn = sqlite3.connect('example.db')
def sql(query: str) -> pd.DataFrame:
    return pd.read_sql_query(query, conn)

In [3]:
sql("""SELECT 
	country_id,
    year,
    gdp,
    -- Show max gdp per country and alias accordingly
	MAX(gdp) OVER (PARTITION BY country_id) AS country_max_gdp
FROM country_stats;""")

,country_id,year,gdp,country_max_gdp
0,1,2000-01-01,NaN,2.061610e+10
1,1,2001-01-01,2.461666e+09,2.061610e+10
2,1,2002-01-01,4.128821e+09,2.061610e+10
3,1,2003-01-01,4.583644e+09,2.061610e+10
4,1,2004-01-01,5.285466e+09,2.061610e+10
...,...,...,...,...
3446,203,2012-01-01,1.424249e+10,1.661996e+10
3447,203,2013-01-01,1.545177e+10,1.661996e+10
3448,203,2014-01-01,1.589105e+10,1.661996e+10
3449,203,2015-01-01,1.630467e+10,1.661996e+10


In [4]:
sql("""SELECT 
	country_id,
    year,
    gdp,
    -- Show max gdp per country and alias accordingly
	MAX(gdp) OVER (PARTITION BY country_id) AS country_max_gdp
FROM country_stats;""")

,country_id,year,gdp,country_max_gdp
0,1,2000-01-01,NaN,2.061610e+10
1,1,2001-01-01,2.461666e+09,2.061610e+10
2,1,2002-01-01,4.128821e+09,2.061610e+10
3,1,2003-01-01,4.583644e+09,2.061610e+10
4,1,2004-01-01,5.285466e+09,2.061610e+10
...,...,...,...,...
3446,203,2012-01-01,1.424249e+10,1.661996e+10
3447,203,2013-01-01,1.545177e+10,1.661996e+10
3448,203,2014-01-01,1.589105e+10,1.661996e+10
3449,203,2015-01-01,1.630467e+10,1.661996e+10


In [5]:
sql("""-- Pull in avg_total_golds by region
SELECT 
	region,
    avg(total_golds) AS avg_total_golds
from
  (SELECT 
      region, 
      country_id, 
      SUM(gold) AS total_golds
  FROM summer_games AS s
  JOIN countries AS c
  ON s.country_id = c.id
  -- Alias the subquery
  GROUP BY region, country_id) AS subquery
GROUP BY region
-- Order by avg_total_golds in descending order
ORDER BY avg_total_golds desc;""")

,region,avg_total_golds
0,NORTHERN AMERICA,42.500000
1,OCEANIA,7.000000
2,LATIN AMER. & CARIB,3.500000
3,ASIA (EX. NEAR EAST),3.250000
4,SUB-SAHARAN AFRICA,3.000000
5,WESTERN EUROPE,2.111111
6,EASTERN EUROPE,1.750000
7,NEAR EAST,1.000000
8,C.W. OF IND. STATES,1.000000
9,NORTHERN AFRICA,NaN


In [7]:
sql("""-- Query region, athlete name, and total_golds
SELECT 
	region,
    athlete_name,
    total_golds
FROM
    (SELECT 
		-- Query region, athlete_name, and total gold medals
        region, 
        name AS athlete_name, 
        SUM(gold) AS total_golds,
        -- Assign a regional rank to each athlete
        ROW_NUMBER() OVER (PARTITION BY region ORDER BY SUM(gold) DESC) AS row_num
    FROM summer_games AS s
    JOIN athletes AS a
    ON a.id = s.athlete_id
    JOIN countries AS c
    ON s.country_id = c.id
    -- Alias as subquery
    GROUP BY region, athlete_name) AS subquery
-- Filter for only the top athlete per region
WHERE row_num = 1;""")

,region,athlete_name,total_golds
0,None,Yusra Mardini,NaN
1,ASIA (EX. NEAR EAST),Kohei Uchimura,2.0
2,BALTICS,ivil Vaiciukeviit,NaN
3,C.W. OF IND. STATES,Oleh Yuriyovych Verniaiev,1.0
4,EASTERN EUROPE,Katinka Hossz,3.0
5,LATIN AMER. & CARIB,Usain St. Leo Bolt,3.0
6,NEAR EAST,Ruth Jebet,1.0
7,NORTHERN AFRICA,Youns Es-Salhi,NaN
8,NORTHERN AMERICA,"Michael Fred Phelps, II",5.0
9,OCEANIA,"Madison ""Madi"" Wilson",1.0


In [8]:
sql("""-- Pull country_gdp by region and country
SELECT 
	region,
    country,
	SUM(gdp) AS country_gdp,
    -- Calculate the global gdp
    SUM(SUM(gdp)) OVER () AS global_gdp,
    -- Calculate percent of global gdp
    SUM(gdp) / SUM(SUM(gdp)) OVER () AS perc_global_gdp,
    -- Calculate percent of gdp relative to its region
    SUM(gdp) / SUM(SUM(gdp)) OVER (PARTITION BY region) AS perc_region_gdp
FROM country_stats AS cs
JOIN countries AS c
ON cs.country_id = c.id
-- Filter out null gdp values
WHERE gdp IS NOT NULL
GROUP BY region, country
-- Show the highest country_gdp at the top
ORDER BY country_gdp DESC;""")

,region,country,country_gdp,global_gdp,perc_global_gdp,perc_region_gdp
0,NORTHERN AMERICA,USA - United States,2.437361e+14,9.714136e+14,2.509087e-01,0.913580
1,ASIA (EX. NEAR EAST),CHN - China,9.051442e+13,9.714136e+14,9.317805e-02,0.354804
2,ASIA (EX. NEAR EAST),JPN - Japan,8.404848e+13,9.714136e+14,8.652183e-02,0.329458
3,WESTERN EUROPE,GER - Germany,5.299421e+13,9.714136e+14,5.455371e-02,0.206898
4,WESTERN EUROPE,GBR - Great Britain,4.205792e+13,9.714136e+14,4.329559e-02,0.164201
...,...,...,...,...,...,...
191,LATIN AMER. & CARIB,CAY - Cayman Islands,3.207033e+09,9.714136e+14,3.301408e-06,0.000046
192,SUB-SAHARAN AFRICA,STP - Sao Tome and Principe,3.135309e+09,9.714136e+14,3.227573e-06,0.000177
193,OCEANIA,MHL - Marshall Islands,2.618646e+09,9.714136e+14,2.695707e-06,0.000139
194,OCEANIA,KIR - Kiribati,2.251355e+09,9.714136e+14,2.317607e-06,0.000120


In [11]:
sql("""-- Bring in region, country, and gdp_per_million
SELECT 
    region,
    country,
    SUM(gdp) / SUM(population) AS gdp_per_million,
    -- Output the worlds gdp_per_million
    SUM(SUM(gdp)) OVER () / SUM(SUM(population)) OVER () AS gdp_per_million_total,
    -- Build the performance_index in the 3 lines below
    (SUM(gdp) / SUM(population))
    /
    (SUM(SUM(gdp)) OVER () / SUM(SUM(population)) OVER ()) AS performance_index
-- Pull from country_stats_clean
FROM country_stats AS cs
JOIN countries AS c 
ON cs.country_id = c.id
-- Filter for 2016 and remove null gdp values
WHERE year = '2016-01-01' AND gdp IS NOT NULL
GROUP BY region, country
-- Show highest gdp_per_million at the top
ORDER BY population DESC;""")

,region,country,gdp_per_million,gdp_per_million_total,performance_index
0,ASIA (EX. NEAR EAST),CHN - China,8117.272869,10213.471203,0.794761
1,ASIA (EX. NEAR EAST),IND - India,1717.474097,10213.471203,0.168158
2,NORTHERN AMERICA,USA - United States,57588.615373,10213.471203,5.638496
3,ASIA (EX. NEAR EAST),INA - Indonesia,3570.282718,10213.471203,0.349566
4,LATIN AMER. & CARIB,BRA - Brazil,8639.370326,10213.471203,0.845880
...,...,...,...,...,...
181,LATIN AMER. & CARIB,SKN - Saint Kitts and Nevis,16596.826574,10213.471203,1.624994
182,OCEANIA,MHL - Marshall Islands,3665.207477,10213.471203,0.358860
183,WESTERN EUROPE,SMR - San Marino,47908.561425,10213.471203,4.690723
184,OCEANIA,PLW - Palau,14077.096219,10213.471203,1.378287


In [16]:
pd.read_csv('./viewers_m_20000.csv').drop_duplicates(subset=['country_id', 'date']).to_sql('web_data', conn, if_exists='replace', index=False)

17721

In [ ]:
SELECT
	-- Pull month and country_id
	DATE_PART('month', date) AS month,
	country_id,
    -- Pull in current month views
    SUM(views) AS month_views,
    -- Pull in last month views
    LAG(SUM(views)) OVER (PARTITION BY country_id ORDER BY DATE_PART('month', date)) AS previous_month_views,
    -- Calculate the percent change
    (SUM(views)/LAG(SUM(views),1) OVER (PARTITION BY country_id ORDER BY DATE_PART('month', date)) - 1) AS perc_change
FROM web_data
WHERE date <= '2018-05-31'
GROUP BY month, country_id;""")

In [22]:
sql("""
SELECT
	-- Pull month and country_id
	STRFTIME('%m', date) AS month,
	country_id,
    -- Pull in current month views
    SUM(views) AS month_views,
    -- Pull in last month views
    LAG(SUM(views)) OVER (PARTITION BY country_id ORDER BY STRFTIME('%m', date)) AS previous_month_views,
    -- Calculate the percent change
    (SUM(views)/LAG(SUM(views),1) OVER (PARTITION BY country_id ORDER BY STRFTIME('%m', date)) - 1) AS perc_change
FROM web_data
WHERE date <= '2021-05-31'
GROUP BY month, country_id;
""")

,month,country_id,month_views,previous_month_views,perc_change
0,01,1,96832,NaN,NaN
1,02,1,76529,96832.0,-1.0
2,03,1,138703,76529.0,0.0
3,04,1,130812,138703.0,-1.0
4,05,1,132106,130812.0,0.0
...,...,...,...,...,...
609,01,123,41873,NaN,NaN
610,02,123,64337,41873.0,0.0
611,03,123,70814,64337.0,0.0
612,04,123,72781,70814.0,0.0


In [23]:
sql("""SELECT
	-- Pull in date and daily_views
	date,
	SUM(views) AS daily_views,
    -- Calculate the rolling 7 day average
	AVG(SUM(views)) OVER (ORDER BY date ROWS BETWEEN 6 PRECEDING AND CURRENT ROW) AS weekly_avg
FROM web_data
GROUP BY date;""")

,date,daily_views,weekly_avg
0,2021-01-10,504309,504309.000000
1,2021-01-11,228540,366424.500000
2,2021-01-12,400534,377794.333333
3,2021-01-13,253392,346693.750000
4,2021-01-14,411323,359619.600000
...,...,...,...
649,2022-10-21,267560,353104.857143
650,2022-10-22,333986,335399.714286
651,2022-10-23,251355,313789.285714
652,2022-10-24,356171,316532.285714


In [24]:
sql("""SELECT 
	-- Pull in date and weekly_avg
	date,
    weekly_avg,
    -- Output the value of weekly_avg from 7 days prior
    LAG(weekly_avg,7) OVER (ORDER BY date) AS weekly_avg_previous,
    -- Calculate percent change vs previous period
    weekly_avg / LAG(weekly_avg,7) OVER (ORDER BY date) - 1 as perc_change
FROM
  (SELECT
      -- Pull in date and daily_views
      date,
      SUM(views) AS daily_views,
      -- Calculate the rolling 7 day average
      AVG(SUM(views)) OVER (ORDER BY date ROWS BETWEEN 6 PRECEDING AND CURRENT ROW) AS weekly_avg
  FROM web_data
  -- Alias as subquery
  GROUP BY date) AS subquery
-- Order by date in descending order
ORDER BY date DESC;""")

,date,weekly_avg,weekly_avg_previous,perc_change
0,2022-10-25,326926.714286,379830.428571,-0.139282
1,2022-10-24,316532.285714,385623.571429,-0.179168
2,2022-10-23,313789.285714,401403.857143,-0.218270
3,2022-10-22,335399.714286,376308.714286,-0.108711
4,2022-10-21,353104.857143,354522.714286,-0.003999
...,...,...,...,...
649,2021-01-14,359619.600000,NaN,NaN
650,2021-01-13,346693.750000,NaN,NaN
651,2021-01-12,377794.333333,NaN,NaN
652,2021-01-11,366424.500000,NaN,NaN


In [25]:
sql("""SELECT
	-- Pull in region and calculate avg tallest height
    region,
    AVG(height) AS avg_tallest,
    -- Calculate region's percent of world gdp
    SUM(gdp)/SUM(SUM(gdp)) OVER () AS perc_world_gdp    
FROM countries AS c
JOIN
    (SELECT 
     	-- Pull in country_id and height
        country_id, 
        height, 
        -- Number the height of each country's athletes
        ROW_NUMBER() OVER (PARTITION BY country_id ORDER BY height DESC) AS row_num
    FROM winter_games AS w 
    JOIN athletes AS a ON w.athlete_id = a.id
    GROUP BY country_id, height
    -- Alias as subquery
    ORDER BY country_id, height DESC) AS subquery
ON c.id = subquery.country_id
-- Join to country_stats
JOIN country_stats AS cs 
ON cs.country_id = c.id
-- Only include the tallest height for each country
WHERE row_num = 1
GROUP BY region;""")

,region,avg_tallest,perc_world_gdp
0,ASIA (EX. NEAR EAST),175.800000,0.262262
1,BALTICS,191.666667,0.001437
2,C.W. OF IND. STATES,182.800000,0.031106
3,EASTERN EUROPE,184.076923,0.021133
4,LATIN AMER. & CARIB,180.000000,0.066241
5,NEAR EAST,175.666667,0.013333
6,NORTHERN AFRICA,170.000000,0.001549
7,NORTHERN AMERICA,194.666667,0.308160
8,OCEANIA,185.000000,0.021306
9,SUB-SAHARAN AFRICA,175.000000,0.000243
